In [1]:
from ipynb.fs.full.UtilitiesConceptDrift import *

In [2]:
# code for loading the entire dataset
dataset = OpenDataset("FullDataset.csv")
print(f"Number of tweets in the dataset: {len(dataset)}")

Number of tweets in the dataset: 33602


In [3]:
# load the initial classifier
clf_init = joblib.load('InitialModel.pkl')

In [3]:
# code for saving the new labelled tweets
# # test_set = test_set.sort_values(by=['likes_count'], ascending=False)
# data = search_tweet_Period(dataset, "2021-04-01", "2021-04-30")
# data.to_csv("ConceptDriftAprile.csv")

In [4]:
# code for loading labelled data
training_febbraio = pd.read_csv("ConceptDriftFebbraio.csv",sep=',',usecols=['timestamp','username','tweet','label'])
training_febbraio = training_febbraio.dropna()

training_marzo = pd.read_csv("ConceptDriftMarzo.csv",sep=',',usecols=['timestamp','username','tweet','label'])
training_marzo = training_marzo.dropna()

training_aprile = pd.read_csv("ConceptDriftAprile.csv",sep=',',usecols=['timestamp','username','tweet','label'])
training_aprile = training_aprile.dropna()

training_initial = pd.read_csv("training_set.csv",sep=',',usecols=['timestamp','username','tweet','label'])

print(f"Number of tweets in training_febbraio: {len(training_febbraio)}")
print(f"Number of tweets in training_marzo: {len(training_marzo)}")
print(f"Number of tweets in training_aprile: {len(training_aprile)}")

Number of tweets in training_febbraio: 60
Number of tweets in training_marzo: 60
Number of tweets in training_aprile: 60


In [5]:
# Merge the all the data
training_init_feb = pd.concat([training_initial,training_febbraio])
training_init_feb_mar = pd.concat([training_init_feb,training_marzo])
#new_trainin_set.label=new_trainin_set.label.astype(int)

print(f"Number of tweets in training_init_feb: {len(training_init_feb)}")
print(f"Number of tweets in training_init_feb_mar: {len(training_init_feb_mar)}")

Number of tweets in training_init_feb: 966
Number of tweets in training_init_feb_mar: 1026


In [6]:
# Training the model with training_init_feb
clf_feb = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=3, analyzer="word", stop_words = set(stopwords.words('italian')), ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf',BernoulliNB()),
])
clf_feb.fit(training_init_feb.tweet, training_init_feb.label)

Pipeline(steps=[('vect',
                 StemmedCountVectorizer(min_df=3,
                                        stop_words={'a', 'abbia', 'abbiamo',
                                                    'abbiano', 'abbiate', 'ad',
                                                    'agl', 'agli', 'ai', 'al',
                                                    'all', 'alla', 'alle',
                                                    'allo', 'anche', 'avemmo',
                                                    'avendo', 'avesse',
                                                    'avessero', 'avessi',
                                                    'avessimo', 'aveste',
                                                    'avesti', 'avete', 'aveva',
                                                    'avevamo', 'avevano',
                                                    'avevate', 'avevi', 'avevo', ...})),
                ('tfidf', TfidfTransformer()), ('clf', BernoulliNB())])

In [7]:
# Training the model with training_init_feb_mar
clf_mar = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=3, analyzer="word", stop_words = set(stopwords.words('italian')), ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf',BernoulliNB()),
])
clf_mar.fit(training_init_feb_mar.tweet, training_init_feb_mar.label)

Pipeline(steps=[('vect',
                 StemmedCountVectorizer(min_df=3,
                                        stop_words={'a', 'abbia', 'abbiamo',
                                                    'abbiano', 'abbiate', 'ad',
                                                    'agl', 'agli', 'ai', 'al',
                                                    'all', 'alla', 'alle',
                                                    'allo', 'anche', 'avemmo',
                                                    'avendo', 'avesse',
                                                    'avessero', 'avessi',
                                                    'avessimo', 'aveste',
                                                    'avesti', 'avete', 'aveva',
                                                    'avevamo', 'avevano',
                                                    'avevate', 'avevi', 'avevo', ...})),
                ('tfidf', TfidfTransformer()), ('clf', BernoulliNB())])

In [13]:
# Save the new models
# CAMBIARE IL NOME!!!!!!!!!!!!!
# joblib.dump(clf_feb, 'modelFebbraio.pkl', compress=9)
# joblib.dump(clf_mar, 'modelMarzo.pkl', compress=9)

['modelMarzo.pkl']

In [9]:
#Do classification with initial model for Marzo
training_marzo["predicted"]= clf_init.predict(training_marzo.tweet)
#print(training_marzo)
benchmark(training_marzo["label"].astype(int), training_marzo["predicted"].astype(int))

accuracy : 0.35
f_score : 0.22510822510822512

[[ 0  0 20]
 [ 0  2 18]
 [ 1  0 19]]
Precision,Recall,f_score
 Classe 0:  [0.         1.         0.33333333] 
 Classe 1:  [0.   0.1  0.95] 
 Classe 2:  [0.         0.18181818 0.49350649] 
 Support:   [20 20 20]


In [10]:
#Do classification with initial model for Aprile
training_aprile["predicted"]= clf_init.predict(training_aprile.tweet)
#print(training_aprile)
benchmark(training_aprile["label"].astype(int), training_aprile["predicted"].astype(int))

accuracy : 0.31666666666666665
f_score : 0.16033755274261602

[[ 0  0 20]
 [ 0  0 20]
 [ 0  1 19]]
Precision,Recall,f_score
 Classe 0:  [0.        0.        0.3220339] 
 Classe 1:  [0.   0.   0.95] 
 Classe 2:  [0.         0.         0.48101266] 
 Support:   [20 20 20]


C:\ProgramData\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
#Do classification with modelFebbraio.pkl for Marzo
predicted = clf_feb.predict(training_marzo.tweet)
benchmark(training_marzo["label"].astype(int), predicted.astype(int))

accuracy : 0.45
f_score : 0.4425837320574162

[[ 7  4  9]
 [ 5 13  2]
 [ 6  7  7]]
Precision,Recall,f_score
 Classe 0:  [0.38888889 0.54166667 0.38888889] 
 Classe 1:  [0.35 0.65 0.35] 
 Classe 2:  [0.36842105 0.59090909 0.36842105] 
 Support:   [20 20 20]


In [25]:
#Do classification with modelMarzo.pkl for Aprile
predicted = clf_mar.predict(training_aprile.tweet)
benchmark(training_aprile["label"].astype(int), predicted.astype(int))

accuracy : 0.5166666666666667
f_score : 0.5017050298380221

[[15  3  2]
 [ 4 10  6]
 [ 7  7  6]]
Precision,Recall,f_score
 Classe 0:  [0.57692308 0.5        0.42857143] 
 Classe 1:  [0.75 0.5  0.3 ] 
 Classe 2:  [0.65217391 0.5        0.35294118] 
 Support:   [20 20 20]
